In [1]:
%matplotlib inline
from PIL import Image
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import os
import itertools
from tqdm import *
from tqdm import tqdm_notebook as tqdm
import random
import numpy as np
import math
import matplotlib.pyplot as plt

from IPython.display import clear_output

from io import StringIO
import re

import pickle
import os

from multiprocessing import Pool 
from itertools import product
import sys
from functools import partial

# Step 1: Load Data Set

In [2]:
fileNames = []
for file in os.listdir("data/train_1"):
    if file.endswith(".jpg"):
        fileNames.append(file)
# fileNamesDF = pd.DataFrame(fileNames)

In [69]:
#only take certain artist so that the probabilities aren't basically all false when doing the tupples
minNumPaintingsPerArtist = 55

# #for each artist, only take n of their paintings
numPaintingsPerAuthor = 100


trainInfo = pd.read_csv('data/train_info.csv')
trainInfo = trainInfo[trainInfo['filename'].isin(fileNames)]

mostPopularArtists = trainInfo['artist'].value_counts()[trainInfo['artist'].value_counts() > minNumPaintingsPerArtist]
# print(mostPopularArtists.head())
# fileNamesDFAll = trainInfo[trainInfo['artist'].isin(mostPopularArtists.index)]
# print(fileNamesDFAll.head())

fileNamesDF = pd.DataFrame()
moreInfo = pd.DataFrame()
for artist in tqdm(mostPopularArtists.index):
    fileNamesDF = fileNamesDF.append(trainInfo[trainInfo['artist'] == artist][:numPaintingsPerAuthor])
    
fileNamesDF.set_index('filename', inplace=True)

In [70]:
# Load the all_data_info for the dataset we are using
allInfo = pd.read_csv('data/all_data_info.csv')
allInfo.set_index('new_filename', inplace=True)

allInfo = allInfo[allInfo.index.isin(fileNamesDF.index)]


featuresDF = pd.DataFrame(index=allInfo.index)
featuresDF['pixelsx'] = allInfo['pixelsx']
featuresDF['pixelsy'] = allInfo['pixelsy']
featuresDF['style'] = allInfo['style']
featuresDF['genre'] = fileNamesDF['genre']


# featuresDF['features'] = pd.Series()

In [72]:
def normalizeSeries(series, maxNum):
    series = series/maxNum
    return series

In [73]:
def featurizeImage(filename):
    # Extract the features from the actual image
    jpgfile = Image.open("data/train_1/" + filename)
#     print(jpgfile)
    
    bands = jpgfile.getbands()
    if bands[0] == 'L':
        # grayscale image
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['grayscale'])
#         return imageDF
#         imageDF['grayscale'] = normalizeSeries(imageDF['grayscale',])
    elif len(bands) == 4:
        #get red, green and blue chanels 
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['red','green','blue','alpha'])
        imageDF['red'] = normalizeSeries(imageDF['red'],255)
        imageDF['green'] = normalizeSeries(imageDF['green'],255)
        imageDF['blue'] = normalizeSeries(imageDF['blue'],255)  
        imageDF['alpha'] = normalizeSeries(imageDF['alpha'],255)  
    elif len(bands) == 3:
        #get red, green and blue chanels 
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['red','green','blue'])
        imageDF['red'] = normalizeSeries(imageDF['red'],255)
        imageDF['green'] = normalizeSeries(imageDF['green'],255)
        imageDF['blue'] = normalizeSeries(imageDF['blue'],255)  
    else:
        imageDF = pd.DataFrame(list(jpgfile.getdata()))
        print(bands)
        return imageDF.mean(), bands
    
    return imageDF.mean()

In [74]:
def pickleDF(fileName,dataFrame,numSplits):
    print(numSplits)
    dfArray = np.array_split(dataFrame, numSplits)
    for i, df in enumerate(dfArray):
        df.to_pickle(fileName+"-"+str(i))

In [86]:
def getImgData(dataFrame,baseName):
    sys.stdout.flush()
#     index = 0
#     while os.path.exists(baseName+"-"+str(index)):
#         index = index + 1
    
    newFileName = baseName + ".pckl"
    print(newFileName)
    sys.stdout.flush()
    featureSeries = []
    for row in tqdm(list(dataFrame.index), leave=False): 
        imageFeatures = featurizeImage(row)
        featureSeries.append(imageFeatures)
        sys.stdout.flush()
    
#     print(featureSeries)
    dataFrame['features'] = featureSeries
#     print(dataFrame['features'])
    dataFrame.to_pickle(newFileName)
    return dataFrame

In [87]:
def concatFrames(dataFrames):
    return pd.concat(dataFrames)

In [88]:
def loadImagesFromPickles(fileNames):
    
    dataFrames = []
    for fileName in fileNames:
        dataFrames.append(pd.read_pickle(fileName))

    return concatFrames(dataFrames)

In [ ]:
baseFileName = 'trainImgsAllcolorBands-NewFeatures'
splitFeaturesData = np.array_split(featuresDF,10)
print(len(splitFeaturesData))
pool = Pool()
inputData = [[splitFeaturesData[i], baseFileName+"-"+str(i)] for i in range(len(splitFeaturesData))]
pool.starmap(getImgData, inputData)
pool.close() 
results = pool.join()

10
trainImgsAllcolorBands-NewFeatures-0.pckl
trainImgsAllcolorBands-NewFeatures-3.pckl
trainImgsAllcolorBands-NewFeatures-2.pckl
trainImgsAllcolorBands-NewFeatures-1.pckl


trainImgsAllcolorBands-NewFeatures-4.pckl


trainImgsAllcolorBands-NewFeatures-5.pckl


trainImgsAllcolorBands-NewFeatures-6.pckl


trainImgsAllcolorBands-NewFeatures-7.pckl


trainImgsAllcolorBands-NewFeatures-8.pckl


trainImgsAllcolorBands-NewFeatures-9.pckl


In [ ]:
print(results)

In [ ]:
pickleLoaded = loadImagesFromPickles([baseFileName+"-"+str(i)+".pckl" for i in range(len(splitFeaturesData))])

In [ ]:
pickleLoaded

# Models

In [ ]:
idx2Categorical